In [ ]:
!mkdir data

In [ ]:
cd data

/content/data


In [ ]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy

--2020-07-29 18:17:18--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113462128 (108M) [application/octet-stream]
Saving to: ‘apple.npy’

apple.npy           100%[===================>] 108.21M  84.9MB/s    in 1.3s    

2020-07-29 18:17:20 (84.9 MB/s) - ‘apple.npy’ saved [113462128/113462128]

--2020-07-29 18:17:24--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 172.253.117.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98055744 (94M) [application/octet-strea

In [ ]:
!ls

apple.npy  banana.npy  grapes.npy  pineapple.npy


In [ ]:
cd ..

/content


In [ ]:
from sklearn.model_selection import train_test_split as tts
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

Using TensorFlow backend.


In [ ]:
# define some constants
N_FRUITS = 4
FRUITS = {0: "Apple", 1: "Banana", 2: "Grape", 3: "Pineapple"}

# number of samples to take in each class
N = 5000

# some other constants
N_EPOCHS = 10

# data files in the same order as defined in FRUITS
files = ["apple.npy", "banana.npy", "grapes.npy", "pineapple.npy"]

In [ ]:
def load(dir, reshaped, files):
    "Load .npy or .npz files from disk and return them as numpy arrays. \
    Takes in a list of filenames and returns a list of numpy arrays."

    data = []
    for file in files:
        f = np.load(dir + file)
        if reshaped:
            new_f = []
            for i in range(len(f)):
                x = np.reshape(f[i], (28, 28))
                x = np.expand_dims(x, axis=0)
                x = np.reshape(f[i], (28, 28, 1))
                new_f.append(x)
            f = new_f
        data.append(f)
    return data


def normalize(data):
    "Takes a list or a list of lists and returns its normalized form"

    return np.interp(data, [0, 255], [-1, 1])


def denormalize(data):
    "Takes a list or a list of lists and returns its denormalized form"

    return np.interp(data, [-1, 1], [0, 255])


def visualize(array):
    "Visulaze a 2D array as an Image"
    array = np.reshape(array, (28,28))
    img = Image.fromarray(array)
    return img


def set_limit(arrays, n):
    "Limit elements from each array up to n elements and return a single list"
    new = []
    for array in arrays:
        i = 0
        for item in array:
            if i == n:
                break
            new.append(item)
            i += 1
    return new


def make_labels(N1, N2):
    "make labels from 0 to N1, each repeated N2 times"
    labels = []
    for i in range(N1):
        labels += [i] * N2
    return labels

In [ ]:
fruits = load("data/", False, ['banana.npy'])

In [ ]:
visualize(fruits[0][90])

In [ ]:

#second argument is True for reshaping the image to a 28x28 form. A conv net expects this format.
fruits = load("data/", True, files)
 
#second argument is False because we don't need to reshape the image. An MLP net expects this format.
#fruits = load("data/", False, files)


# limit no of samples in each class to N
fruits = set_limit(fruits, N)

# normalize the values
fruits = map(normalize, fruits)

# define the labels
labels = make_labels(N_FRUITS, N)

# prepare the data
x_train, x_test, y_train, y_test = tts(fruits, labels, test_size=0.05)

# one hot encoding
Y_train = np_utils.to_categorical(y_train, N_FRUITS)
Y_test = np_utils.to_categorical(y_test, N_FRUITS)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(N_FRUITS, activation='softmax'))

In [ ]:
# model = Sequential()
# model.add(Dense(units=600, activation='relu', input_dim=784))
# model.add(Dropout(0.3))
# model.add(Dense(units=400, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(units=100, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(units=25, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(units=N_FRUITS, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# train
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)

print "Training complete"

print "Evaluating model"
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
    if np.argmax(preds[i]) == y_test[i]:
        score += 1

print "Accuracy: ", ((score + 0.0) / len(preds)) * 100


model.save("fruits"+ ".h5")
print "Model saved"


Epoch 1/10
19000/19000 [==============================] - 10s 529us/step - loss: 0.3160 - accuracy: 0.8958
Epoch 2/10
19000/19000 [==============================] - 3s 155us/step - loss: 0.1885 - accuracy: 0.9414
Epoch 3/10
19000/19000 [==============================] - 3s 156us/step - loss: 0.1504 - accuracy: 0.9541
Epoch 4/10
19000/19000 [==============================] - 3s 160us/step - loss: 0.1237 - accuracy: 0.9607
Epoch 5/10
19000/19000 [==============================] - 3s 155us/step - loss: 0.1007 - accuracy: 0.9659
Epoch 6/10
19000/19000 [==============================] - 3s 154us/step - loss: 0.0874 - accuracy: 0.9704
Epoch 7/10
19000/19000 [==============================] - 3s 155us/step - loss: 0.0720 - accuracy: 0.9748
Epoch 8/10
19000/19000 [==============================] - 3s 154us/step - loss: 0.0636 - accuracy: 0.9789
Epoch 9/10
19000/19000 [==============================] - 3s 154us/step - loss: 0.0529 - accuracy: 0.9824
Epoch 10/10
19000/19000 [====================